## 진행 사항 공유
#### 05.26
> 배 -> 엄
1. 파일 전체가 아니라 일부만 read 
2. 이하 셀에 vs code 에서 한 내용만 일단 복사 해놓음

#### 05.27
> 엄
1. 진호오빠가 해준 컬럼 삭제 순서 조금 수정해놓음  
>> - 레이블 데이터인 TARGET 컬럼 label이라는 변수에 따로 빼놓음  
>> - 컬럼 인덱스로 삭제할 때 컬럼명 지정 삭제 전에 먼저 삭제(컬럼 순서 바뀌므로)
2. 26일에 진행한 결측치 제거, DAYS_EMPLOYED 값 변환 순서 정리, 설명 작성
3. CODE_GENDER XNA데이터 4개 삭제
4. DAYS_LAST_PHONE_CHANGE 결측치 1개 삭제, 양수 및 년단위 변환

#### 05.28
> 황
1. '사용하지 않는 컬럼 제거' 에서 
>> - col_drop1 = df_origin.columns[40:44]을 [40:43]으로 변경
>> - col_drop2 = df_origin.columns[44:94]을 [43:94]으로 변경
2. 나이(DAYS_BIRTH)보다 거주일수(DAYS_REGISTRATION)가 많은 경우 삭제
3. 양수 및 년단위 변환
>> - DAYS_REGISTRATION
>> - DAYS_BIRTH
>> - DAYS_ID_PUBLISH
5. AMT_ANNUITY, AMT_GOODS_PRICE 결측치 삭제
6. NAME_FAMILY_STATUS에 Unknown 값 자동 삭제된 것 확인
7. CNT_FAM_MEMBERS 결측치 자동 삭제된 것 확인
8. DAYS_ID_PUBLISH_int, DAYS_BIRTH_int, DAYS_REGISTRATION_int 파생변수 만듬

**df['OCCUPATION_TYPE']의 unknown값(41019개) 사실상 결측치임. 어떻게 할 지 고민하고 나서 인코딩 필요**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 드라이브 마운트

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

### 데이터 불러오기

In [4]:
# 진호용
# df_origin = pd.read_csv("/content/drive/MyDrive/Hackathon_ubion/application_data.csv")

# 지우용, 세진용
df_origin = pd.read_csv("./datasets/application_data.csv")
df_origin.head(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# 원본 복구를 위해 copy()해놓음
# df로 작업하고 원본과 비교 필요 시 df_origin으로 확인

df = df_origin.copy()

# print(df.shape)
df.shape

(307511, 122)

In [6]:
df.info(verbose=True, null_counts=True)

C:\Users\zino0\AppData\Local\Temp\ipykernel_32348\2612161665.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose=True, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 122 columns):
 #    Column                        Non-Null Count   Dtype  
---   ------                        --------------   -----  
 0    SK_ID_CURR                    307511 non-null  int64  
 1    TARGET                        307511 non-null  int64  
 2    NAME_CONTRACT_TYPE            307511 non-null  object 
 3    CODE_GENDER                   307511 non-null  object 
 4    FLAG_OWN_CAR                  307511 non-null  object 
 5    FLAG_OWN_REALTY               307511 non-null  object 
 6    CNT_CHILDREN                  307511 non-null  int64  
 7    AMT_INCOME_TOTAL              307511 non-null  float64
 8    AMT_CREDIT                    307511 non-null  float64
 9    AMT_ANNUITY                   307499 non-null  float64
 10   AMT_GOODS_PRICE               307233 non-null  float64
 11   NAME_TYPE_SUITE               306219 non-null  object 
 12   NAME_INCOME_TYPE            

In [7]:
df['AMT_REQ_CREDIT_BUREAU_HOUR'].value_counts()

0.0    264366
1.0      1560
2.0        56
3.0         9
4.0         1
Name: AMT_REQ_CREDIT_BUREAU_HOUR, dtype: int64

In [8]:
264366/307511

0.8596960759127316

In [9]:
df['AMT_REQ_CREDIT_BUREAU_HOUR']

0         0.0
1         0.0
2         0.0
3         NaN
4         0.0
         ... 
307506    NaN
307507    NaN
307508    1.0
307509    0.0
307510    0.0
Name: AMT_REQ_CREDIT_BUREAU_HOUR, Length: 307511, dtype: float64

In [10]:
df['AMT_REQ_CREDIT_BUREAU_DAY'].value_counts()

0.0    264503
1.0      1292
2.0       106
3.0        45
4.0        26
5.0         9
6.0         8
9.0         2
8.0         1
Name: AMT_REQ_CREDIT_BUREAU_DAY, dtype: int64

In [11]:
# 결측치 확인
df.isna().sum()

SK_ID_CURR                        0
TARGET                            0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     41519
AMT_REQ_CREDIT_BUREAU_WEEK    41519
AMT_REQ_CREDIT_BUREAU_MON     41519
AMT_REQ_CREDIT_BUREAU_QRT     41519
AMT_REQ_CREDIT_BUREAU_YEAR    41519
Length: 122, dtype: int64

### 사용하지 않는 컬럼 제거

In [12]:
# 컬럼명 지정해서 삭제하기 전(column index 순서 바뀌기 전) column index로 뭉텅이 삭제

# EXT_SOURCE 1~3 -> 의미파악불가
col_drop1 = df_origin.columns[41:44]
df.drop(col_drop1, axis=1, inplace=True)
print(df.columns)

# 주거 관련 columns  -> 의미파악불가 
col_drop2 = df_origin.columns[44:95]
df.drop(col_drop2, axis=1, inplace=True)

# FLAG_DOCUMENT_2~21 -> 의미파악 불가 
col_drop3 = df_origin.columns[96:116]
df.drop(col_drop3, axis=1, inplace=True)


print(df.shape)
df.head()


Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=119)
(307511, 48)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,Business Entity Type 3,-1134.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,School,-828.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,Government,-815.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,Business Entity Type 3,-617.0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,1,1,Religion,-1106.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df.drop(['SK_ID_CURR', 
         'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE','FLAG_PHONE',
         'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 
         'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 
         'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY' ], axis=1, inplace=True)

In [14]:
df.columns

Index(['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_EMAIL',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE',
       'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object')

### DAYAS_EMPLOYED 가 양수인 데이터 의미 찾기

In [15]:
# DAYS_EMPLOYED(근속일수) 데이터 양수인 데이터 개수
len(df.loc[df['DAYS_EMPLOYED'] > 0])

55374

In [16]:
# ORGANIZATION_TYPE 데이터 유형 중 XNA인 데이터 발견, 그 수가 DAYS_EMPLOYED(근속일수) 데이터 양수인 데이터 개수와 동일

df['ORGANIZATION_TYPE'].value_counts()

Business Entity Type 3    67992
XNA                       55374
Self-employed             38412
Other                     16683
Medicine                  11193
Business Entity Type 2    10553
Government                10404
School                     8893
Trade: type 7              7831
Kindergarten               6880
Construction               6721
Business Entity Type 1     5984
Transport: type 4          5398
Trade: type 3              3492
Industry: type 9           3368
Industry: type 3           3278
Security                   3247
Housing                    2958
Industry: type 11          2704
Military                   2634
Bank                       2507
Agriculture                2454
Police                     2341
Transport: type 2          2204
Postal                     2157
Security Ministries        1974
Trade: type 2              1900
Restaurant                 1811
Services                   1575
University                 1327
Industry: type 7           1307
Transpor

-> DAYS_EMPLOYED(근속일수) 데이터 양수이고 ORGANIZATION_TYPE(근무지 유형) 데이터가 "XNA"인 데이터를 무직인 것으로 판단

### OCCUPATION_TYPE 결측치 대체  

> 들여쓴 블록 DAYS_EMPLOYED 가 양수이고 ORGANIZATION_TYPE 이 XNA인 데이터 : 무직


> 들여쓴 블록 그 외의 결측치 : 미응답자(근무지 유형만 답변하고 직무는 답변하지 않음)

In [17]:
df['OCCUPATION_TYPE'].value_counts()

Laborers                 55186
Sales staff              32102
Core staff               27570
Managers                 21371
Drivers                  18603
High skill tech staff    11380
Accountants               9813
Medicine staff            8537
Security staff            6721
Cooking staff             5946
Cleaning staff            4653
Private service staff     2652
Low-skill Laborers        2093
Waiters/barmen staff      1348
Secretaries               1305
Realty agents              751
HR staff                   563
IT staff                   526
Name: OCCUPATION_TYPE, dtype: int64

In [18]:
# OCCUPATION_TYPE(직무 유형) 데이터 결측치 개수
len(df.loc[df["OCCUPATION_TYPE"].isna()]) #96391/307511*100 = 31%

96391

In [19]:
# DAYS_EMPLOYED 가 양수인 OCCUPATION_TYPE 데이터 개수
len(df.loc[df['DAYS_EMPLOYED'] > 0,'OCCUPATION_TYPE'])

55374

In [20]:
# DAYS_EMPLOYED 가 양수이면서 ORGANIZATION_TYPE 이 XNA인 OCCUPATION_TYPE 데이터 개수
len(df.loc[(df['DAYS_EMPLOYED'] > 0) & (df['ORGANIZATION_TYPE'] == "XNA"),'OCCUPATION_TYPE'])

55374

#### 무직(DAYS_EMPLOYED 가 양수이면서 ORGANIZATION_TYPE 이 XNA)인 OCCUPATION_TYPE 데이터 채우기 위해 재확인

In [21]:
# DAYS_EMPLOYED 가 양수인 데이터에 대한 OCCUPATION_TYPE 데이터 유형 확인
df.loc[df['DAYS_EMPLOYED'] > 0,'OCCUPATION_TYPE'].unique()

array([nan, 'Cleaning staff'], dtype=object)

In [22]:
# DAYS_EMPLOYED 가 양수인 데이터에 대한 OCCUPATION_TYPE 데이터가 Cleaning staff인 개수 확인
len(df.loc[(df['DAYS_EMPLOYED'] > 0 )& (df['OCCUPATION_TYPE']=="Cleaning staff")])

2

-> DAYS_EMPLOYED 가 양수인 데이터에 대한 OCCUPATION_TYPE 데이터 유형과 그 개수 파악

-> OCCUPATION_TYPE 이 nan으로만 구성되어 있을 것이라고 예상했지만 Cleaning staff 데이터가 2개 존재 -> 이상치라고 판단 (4653개 중에 2개라)

### 결측치 제거

In [23]:
df.drop((df.loc[(df['DAYS_EMPLOYED'] > 0)&(df['OCCUPATION_TYPE']=='Cleaning staff')].index),axis=0,inplace=True)

## df.drop((df.loc[(df['DAYS_EMPLOYED'] > 0) & (df['ORGANIZATION_TYPE'] == "XNA")& (df['OCCUPATION_TYPE']=="Cleaning staff")])

#   df['OCCUPATION_TYPE'] 결측치에서 df['DAYS_EMPLOYED'] > 0) & (df['ORGANIZATION_TYPE'] == 'XNA') 인 사람들 'Unemployed' 로 바꿀건데
#   df['DAYS_EMPLOYED'] > 0 인 사람들이랑 df['ORGANIZATION_TYPE'] == 'XNA' 사람들 같은 사람들이라서 바꿀 때 하나만 조건으로 해도 됨

In [24]:
# 이상치 삭제 잘 됐는지 확인
len(df.loc[(df['DAYS_EMPLOYED'] > 0 )& (df['OCCUPATION_TYPE']=="Cleaning staff")])

0

In [25]:
len(df.loc[df['DAYS_EMPLOYED'] > 0 ])

55372

In [26]:
len(df.loc[df['ORGANIZATION_TYPE'] == "XNA"])

55372

In [27]:
# 결측치 0으로 대체
df['OCCUPATION_TYPE'].fillna(0,inplace=True)

In [28]:
# DAYS_EMPLOYED 가 양수이면서 OCCUPATION_TYPE 이 nan이었던 데이터에 대해 Unemployed로 대체
df.loc[(df['DAYS_EMPLOYED'] > 0)&(df['OCCUPATION_TYPE']==0),'OCCUPATION_TYPE']='Unemployed'

In [29]:
# OCCUPATION_TYPE 이 nan이었던 그 외 데이터 Unknown으로 대체
df['OCCUPATION_TYPE'].replace(0,'Unknown',inplace=True)

In [30]:
df['OCCUPATION_TYPE'].value_counts()

Unemployed               55372
Laborers                 55186
Unknown                  41019
Sales staff              32102
Core staff               27570
Managers                 21371
Drivers                  18603
High skill tech staff    11380
Accountants               9813
Medicine staff            8537
Security staff            6721
Cooking staff             5946
Cleaning staff            4651
Private service staff     2652
Low-skill Laborers        2093
Waiters/barmen staff      1348
Secretaries               1305
Realty agents              751
HR staff                   563
IT staff                   526
Name: OCCUPATION_TYPE, dtype: int64

결측치 제거 수행 완료  
데이터 생성 시점으로부터 과거 일자를 계산한 값으로 음수값이었던 DAYS_EMPLOYED 데이터 양수로 바꿔줌과 동시에 무직인 사람 데이터는 그 값을 0으로 바꿔주고 기존 근속일수가 존재했던 사람에 대해서는 +1 수행(무직인 사람과 구별하기 위해)

In [31]:
df['DAYS_EMPLOYED'] = np.where(df['DAYS_EMPLOYED'] <= 0, -(df['DAYS_EMPLOYED'])+1, 0)

**df['OCCUPATION_TYPE']의 unknown값(41019개) 사실상 결측치임. 어떻게 할 지 고민하고 나서 인코딩 필요**

In [32]:
len(df.loc[df["CODE_GENDER"]=="XNA"])

4

In [33]:
# CODE_GENDER XNA데이터(4개) 삭제
df.drop(df.loc[df["CODE_GENDER"]=="XNA"].index, axis=0, inplace=True)

In [34]:
df.loc[df["CODE_GENDER"]=="XNA"]

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR


In [35]:
df.columns

Index(['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_EMAIL',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE',
       'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object')

In [36]:
# DAYS_LAST_PHONE_CHANGE 결측치(1개) 삭제
df.drop(df.loc[df["DAYS_LAST_PHONE_CHANGE"].isna()].index, axis=0, inplace=True)

In [37]:
df.loc[df["DAYS_LAST_PHONE_CHANGE"].isna()]

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR


In [38]:
df["DAYS_LAST_PHONE_CHANGE"].head(3)

0   -1134.0
1    -828.0
2    -815.0
Name: DAYS_LAST_PHONE_CHANGE, dtype: float64

In [39]:
# DAYS_LAST_PHONE_CHANGE 양수 및 년단위 변환
df["DAYS_LAST_PHONE_CHANGE"] = df["DAYS_LAST_PHONE_CHANGE"].apply(lambda x:(-x)/365)
df["DAYS_LAST_PHONE_CHANGE"].head(3)

0    3.106849
1    2.268493
2    2.232877
Name: DAYS_LAST_PHONE_CHANGE, dtype: float64

In [40]:
df["DAYS_REGISTRATION"].head(3)

0   -3648.0
1   -1186.0
2   -4260.0
Name: DAYS_REGISTRATION, dtype: float64

In [41]:
df.loc[(-df["DAYS_REGISTRATION"] > -df["DAYS_BIRTH"]),['DAYS_REGISTRATION','DAYS_BIRTH']]

,DAYS_REGISTRATION,DAYS_BIRTH
266366,-10116.041667,-10116


In [42]:
# 나이(DAYS_BIRTH)보다 거주일수(DAYS_REGISTRATION)가 많은 경우 삭제
df.drop(df.loc[(-df["DAYS_REGISTRATION"] > -df["DAYS_BIRTH"])].index, axis=0, inplace=True)
df.loc[(-df["DAYS_REGISTRATION"] > -df["DAYS_BIRTH"])]

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR


In [43]:
# DAYS_REGISTRATION 양수 및 년단위 변환
df["DAYS_REGISTRATION"] = df["DAYS_REGISTRATION"].apply(lambda x:(-x)/365)
df["DAYS_REGISTRATION"].head(3)

0     9.994521
1     3.249315
2    11.671233
Name: DAYS_REGISTRATION, dtype: float64

In [44]:
df["DAYS_BIRTH"].head(3)

0    -9461
1   -16765
2   -19046
Name: DAYS_BIRTH, dtype: int64

In [45]:
# DAYS_BIRTH 양수 및 년단위 변환
df["DAYS_BIRTH"] = df["DAYS_BIRTH"].apply(lambda x:(-x)/365)
df["DAYS_BIRTH"].head(3)

0    25.920548
1    45.931507
2    52.180822
Name: DAYS_BIRTH, dtype: float64

In [46]:
df["DAYS_ID_PUBLISH"].head(3)

0   -2120
1    -291
2   -2531
Name: DAYS_ID_PUBLISH, dtype: int64

In [47]:
# DAYS_ID_PUBLISH 양수 및 년단위 변환
df["DAYS_ID_PUBLISH"] = df["DAYS_ID_PUBLISH"].apply(lambda x:(-x)/365)
df["DAYS_ID_PUBLISH"].head(3)

0    5.808219
1    0.797260
2    6.934247
Name: DAYS_ID_PUBLISH, dtype: float64

In [48]:
df["DAYS_ID_PUBLISH"].value_counts()

11.104110    169
11.219178    162
11.084932    161
12.101370    159
11.660274    158
            ... 
16.153425      1
16.221918      1
15.761644      1
16.328767      1
17.016438      1
Name: DAYS_ID_PUBLISH, Length: 6168, dtype: int64

In [49]:
df["DAYS_ID_PUBLISH_int"] = df["DAYS_ID_PUBLISH"].apply(lambda x:int(x)) # 정수로 바꿔 신분증발급연차
df["DAYS_BIRTH_int"] = df["DAYS_BIRTH"].apply(lambda x:int(x)) # 나이를 정수로
df.groupby(["DAYS_ID_PUBLISH_int",'DAYS_BIRTH_int'])['DAYS_BIRTH_int'].count() # 신분증발급연차 별 나이

df["DAYS_REGISTRATION_int"] = df["DAYS_REGISTRATION"].apply(lambda x:int(x))

In [52]:
pd.crosstab(df.DAYS_ID_PUBLISH_int, df.DAYS_BIRTH_int, margins=True).style.background_gradient(cmap='summer_r')
# 20대가 초기 발급인 것 같고 40대 후반에서 50대 초반이 재발급인 것 같은데 20대보다 40&50대가 많은 걸 알 수 있음

ImportError: Pandas requires version '3.0.0' or newer of 'jinja2' (version '2.11.0' currently installed).

'pandas'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [53]:
df.loc[df["DAYS_ID_PUBLISH_int"] ==0]["DAYS_BIRTH_int"].value_counts()

45    6973
46    1067
27     386
28     351
31     305
30     294
29     288
34     267
33     258
32     243
35     240
21     224
37     219
36     218
38     216
23     207
39     204
41     195
26     193
40     192
25     184
22     181
24     176
42     175
43     128
47     125
44     101
52      95
48      93
53      86
59      81
60      81
51      78
54      76
50      75
56      71
55      71
57      69
49      65
58      65
61      62
62      60
63      47
64      43
65      23
67      23
66      23
68       9
20       1
Name: DAYS_BIRTH_int, dtype: int64

In [54]:
len(df.loc[df["AMT_ANNUITY"].isna()])

12

In [55]:
# AMT_ANNUITY 결측치(12개) 삭제
df.drop(df.loc[df["AMT_ANNUITY"].isna()].index, axis=0, inplace=True)
df.loc[df["AMT_ANNUITY"].isna()]

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_ID_PUBLISH_int,DAYS_BIRTH_int,DAYS_REGISTRATION_int


In [56]:
len(df.loc[df["AMT_GOODS_PRICE"].isna()])

276

In [57]:
# AMT_ANNUITY 결측치(276개) 삭제
df.drop(df.loc[df["AMT_GOODS_PRICE"].isna()].index, axis=0, inplace=True)
df.loc[df["AMT_GOODS_PRICE"].isna()]

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_ID_PUBLISH_int,DAYS_BIRTH_int,DAYS_REGISTRATION_int


In [58]:
df["NAME_FAMILY_STATUS"].value_counts()

Married                 196253
Single / not married     45383
Civil marriage           29748
Separated                19757
Widow                    16074
Name: NAME_FAMILY_STATUS, dtype: int64

In [59]:
df_origin["NAME_FAMILY_STATUS"].value_counts() 

Married                 196432
Single / not married     45444
Civil marriage           29775
Separated                19770
Widow                    16088
Unknown                      2
Name: NAME_FAMILY_STATUS, dtype: int64

df_origin의 "NAME_FAMILY_STATUS" 에 unknown이 2개 있었는데 위에서부터 drop을 하면서 2개가 사라짐 => 따라서 unknown에 대해 drop 필요 x

In [60]:
len(df.loc[df["CNT_FAM_MEMBERS"].isna()])

0

In [61]:
len(df_origin.loc[df_origin["CNT_FAM_MEMBERS"].isna()])

2

df_origin의 "CNT_FAM_MEMBERS" 에 결측치 2개 있었는데 위에서부터 drop을 하면서 2개가 사라짐 => 따라서 결측치 처리 필요 x

In [62]:
df['AMT_INCOME_TOTAL'].head(3)

0    202500.0
1    270000.0
2     67500.0
Name: AMT_INCOME_TOTAL, dtype: float64

### 머신러닝 사전준비

In [63]:
# 피처 선정 시 사용하지 않는 TARGET 컬럼 제외한 데이터프레임으로 전처리 수행
label = df["TARGET"]
df = df.drop("TARGET", axis=1)

In [64]:
# from sklearn.decomposition import PCA
# X_1= X.loc[:,'APARTMENTS_AVG':'NONLIVINGAREA_MEDI']
# pca = PCA(n_components=2)
# pca_transformed = pca.fit_transform(X_1)

# print(pca_transformed)
# df['pca_x'] = pca_transformed[:,0]
# df['pca_y'] = pca_transformed[:,1]
# df.head()

In [65]:
# marker0_ind = df[df['TARGET']==0].index
# marker1_ind = df[df['TARGET']==1].index

# plt.scatter(x=df.loc[marker0_ind, 'pca_x'], y=df.loc[marker0_ind, 'pca_y'] , marker='o')
# plt.scatter(x=df.loc[marker1_ind, 'pca_x'], y=df.loc[marker1_ind, 'pca_y'] , marker='s')


# plt.xlabel('PCA_1')
# plt.ylabel('PCA_2')
# plt.show()

In [66]:
## 이상치의 값들을 출력하는 함수 생성
## 매개변수는 2개 데이터프레임, 컬럼의 이름
def outliers_iqr(df, col):
    ## 1사분위수, 3사분위수 변수에 대입
    quartile_1, quartile_3 = np.percentile(df[col], [25, 75]) #25프로, 75프로
    ## iqr 값을 대입
    iqr = quartile_3 - quartile_1
    ## 하단 경계 값, 상단 경계 값
    lower_whis = quartile_1 - (1.5*iqr)
    upper_whis = quartile_3 + (1.5*iqr)
    print(lower_whis, upper_whis)
    ## 데이터프레임에서 상단의 경계보다 크거나 하단의 경계보다 작은 데이터를 출력
    outliers = df[(df[col] > upper_whis) | (df[col] < lower_whis)]
    return outliers[[col]]

In [67]:
## 이상치를 확인하는 함수를 호출
outliers = outliers_iqr(df, 'CNT_FAM_MEMBERS')
outliers

0.5 4.5


,CNT_FAM_MEMBERS
91,5.0
92,5.0
144,5.0
180,5.0
182,5.0
...,...
307293,6.0
307329,5.0
307387,5.0
307462,5.0


In [68]:
4000/307511

0.01300766476646364

In [69]:
# DAYS_LAST_PHONE_CHANGE 피처와 AMT_INCOME_TOTAL 피처 상관관계 확인
# corr_matrix = df[['DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL']].corr()
# corr_matrix


In [70]:
# 루피 -> 원화 

df["AMT_INCOME_TOTAL"] = df["AMT_INCOME_TOTAL"].apply(lambda x:int(x*16.08))
df["AMT_INCOME_TOTAL"]

0         3256199
1         4341600
2         1085400
3         2170800
4         1953719
           ...   
307506    2532599
307507    1157759
307508    2460239
307509    2749679
307510    2532599
Name: AMT_INCOME_TOTAL, Length: 307215, dtype: int64

In [77]:
# One-hot encoding 하는 컬럼 unique 값 확인 후, 
    # 5 초과시 -> binary encoding 으로 변경(노션도 수정)

# 5이하 
#   df['NAME_CONTRACT_TYPE'].unique()
#   df['CODE_GENDER'].unique()
#   df['FLAG_OWN_CAR'].unique()
#   df['FLAG_OWN_REALTY'].unique()
#   df['NAME_FAMILY_STATUS'].unique()
# 5초과 
df['NAME_INCOME_TYPE'].unique()

df['DAYS_BIRTH'].unique()

array([25.92054795, 45.93150685, 52.18082192, ..., 21.78356164,
       21.5260274 , 68.66027397])

In [78]:
# days_birth

df['DAYS_BIRTH_int'].head()

0    25
1    45
2    52
3    52
4    54
Name: DAYS_BIRTH_int, dtype: int64

In [103]:
df['DAYS_BIRTH_int_n0s']=""


In [104]:
df.loc[(df['DAYS_BIRTH_int']>=20) & (df['DAYS_BIRTH_int'] <30),"DAYS_BIRTH_INT_n0s"]="20s"

In [105]:
df.loc[(df['DAYS_BIRTH_int']>=30) & (df['DAYS_BIRTH_int'] <40),"DAYS_BIRTH_INT_n0s"]="30s"

In [106]:
df.loc[(df['DAYS_BIRTH_int']>=40) & (df['DAYS_BIRTH_int'] <50),"DAYS_BIRTH_INT_n0s"]="40s"

In [107]:
df.loc[(df['DAYS_BIRTH_int']>=50) & (df['DAYS_BIRTH_int'] <60),"DAYS_BIRTH_INT_n0s"]="50s"

In [108]:
df.loc[(df['DAYS_BIRTH_int']>=60),"DAYS_BIRTH_INT_n0s"]="60s"

In [110]:
df["DAYS_BIRTH_INT_n0s'"].value_counts()

30s    82203
40s    76478
50s    68037
20s    44918
60s    35579
Name: DAYS_BIRTH_INT_n0s', dtype: int64

In [97]:
df.loc[(df['DAYS_BIRTH_int']>=20) & (df['DAYS_BIRTH_int'] <30),"DAYS_BIRTH_INT_n0s'"]

0         20s
10        20s
15        20s
17        20s
28        20s
         ... 
307470    20s
307480    20s
307488    20s
307490    20s
307506    20s
Name: DAYS_BIRTH_INT_n0s', Length: 44918, dtype: object

In [86]:
df['DAYS_BIRTH_int_n0s'] = df.loc[(df['DAYS_BIRTH_int']>=20) & (df['DAYS_BIRTH_int'] <30)]['DAYS_BIRTH_int']
df['DAYS_BIRTH_int_n0s'] = df.loc[(df['DAYS_BIRTH_int']>=30) & (df['DAYS_BIRTH_int'] <40)]['DAYS_BIRTH_int']
df['DAYS_BIRTH_int_n0s'] = df.loc[(df['DAYS_BIRTH_int']>=40) & (df['DAYS_BIRTH_int'] <50)]['DAYS_BIRTH_int']
df['DAYS_BIRTH_int_n0s'] = df.loc[(df['DAYS_BIRTH_int']>=50) & (df['DAYS_BIRTH_int'] <60)]['DAYS_BIRTH_int']
df['DAYS_BIRTH_int_n0s'] = df.loc[(df['DAYS_BIRTH_int']>=60)]['DAYS_BIRTH_int']     
df['DAYS_BIRTH_int_n0s']

0     25.0
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10    27.0
Name: DAYS_BIRTH_int_n0s, dtype: float64

In [87]:
df['DAYS_BIRTH_int_n0s'] = df.loc[(df['DAYS_BIRTH_int']>=30) & (df['DAYS_BIRTH_int'] <40)]['DAYS_BIRTH_int']  

In [88]:
df['DAYS_BIRTH_int_n0s'] = df.loc[(df['DAYS_BIRTH_int']>=40) & (df['DAYS_BIRTH_int'] <50)]['DAYS_BIRTH_int']  

In [89]:
df['DAYS_BIRTH_int_n0s'] = df.loc[(df['DAYS_BIRTH_int']>=50) & (df['DAYS_BIRTH_int'] <60)]['DAYS_BIRTH_int']  

In [90]:
df['DAYS_BIRTH_int_n0s'] = df.loc[(df['DAYS_BIRTH_int']>=60)]['DAYS_BIRTH_int']  

In [102]:
df['DAYS_BIRTH_int_n0s'].value_counts()

    307215
Name: DAYS_BIRTH_int_n0s, dtype: int64